# OpenStreetMap Data Cleaning

by Johannes Bock, September 18, 2016

In the following project I will choose an area of the world in https://www.openstreetmap.org and use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data. Moreover, after thoroughly auditing and cleaning the dataset, I will build my own local SQL database, import the OSM data and explore the database by running some SQL queries.

### Map Area

In the following project I chose to look at OSM data for my hometown ([Karlsruhe, Germany](http://www.openstreetmap.org/export#map=13/49.0118/8.4131)) and investigated the quality of the available information on Karlsruhe in OSM.

### Problems Encountered in the Map

First, I downloaded and audied the OSM data for Karlsruhe. In general, the OSM data for my hometown already is quite clean, since I could not find any problems with inconsistent streetnames or postcodes, for instance. However, I noticed three problems with the data, which I will discuss in the following order:

1) OSM Keys with unexpected format <br>
2) Inconsistent formatting of phone numbers <br>
3) Inconsistent names for a local company <br>
4) Inconsistent date formats in OSM key = "start_date"

#### Keys with unexpected format
Looking at some keys in the dataset, I have found 16342 entries with an unexpected format. 

In [ ]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [ ]:
#Result
{'lower': 344630, 'lower_colon': 242823, 'other': 16342, 'problemchars': 0}

Looking at some examples which were labeled "other", I could see some patterns here:

In [ ]:
<tag k="railway:signal:crossing_distant" v="DE-ESO:bü2" />
<tag k="parking:condition:left" v="free" />
<tag k="traffic_sign:backward" v="none" />
<tag k="ref:hafas:DB" v="723635" />

Researching these keys reveals that they seem to be very specialized keys for describing ways in OSM and sometimes they are not even part of the internationally standardized OSM key definitions (e.g. [country-specific signal tagging](http://wiki.openstreetmap.org/wiki/Tag:railway%3Dsignal)). Since these keys are not relevant for the subsequent analysis and do not contain meaningful information for my purposes, I will just ignore these highly specialized keys and will not add them to the database. 

#### Inconsistent formatting of phone numbers
In the OSM dataset there are some datafields containing phone numbers. These include the OSM keys "phone" and "contact:phone". Therefore, I decided to audit these fields and check whether there are inconsistencies. The following code describes my analysis of the data.

In [1]:
import xml.etree.cElementTree as ET
import re
import pprint

def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            
def audit_phone(element,count):
    phone_pattern = re.compile(r"^(\+49)\s[0-9]*\s[0-9]*$",re.I)
    whitespace_end = re.compile(r"^(\+49)\s[0-9]*\s[0-9\s|\-]*$")
    whitespace_start = re.compile(r"^(\+49)[\-|\s*][0-9]*[\-|\s*][0-9]*$")
    no_country_code = re.compile(r"^([0-9])*\s[0-9]*$",re.I)
    numbers_only = re.compile(r"^[0-9]*$",re.I)
    if "phone" in element.attrib["k"]:
        if phone_pattern.search(element.attrib["v"]):
            count["Expected_format"]+=1
        elif whitespace_end.search(element.attrib["v"]) and not phone_pattern.search(element.attrib["v"]):
            count["Whitespace_problem"]+=1
        elif whitespace_start.search(element.attrib["v"]) and not phone_pattern.search(element.attrib["v"]):         
            count["Whitespace_problem"]+=1
        elif no_country_code.search(element.attrib["v"]):
            count["no_country_code"] += 1
        elif numbers_only.search(element.attrib["v"]):
            count["numbers_only"]+=1
            #return element.attrib["v"]
        else:
            count["other"] += 1             
            return element.attrib["v"]
        
def process_phone(filename,tags=("tag")):
    unexpected_phone_numbers = set()
    count = {"Expected_format": 0 , "Whitespace_problem":0, "no_country_code":0, "numbers_only":0,"other":0}
    for element in get_element(filename, tags):
        number = audit_phone(element,count)
        unexpected_phone_numbers.add(number)
    return unexpected_phone_numbers, count

data = "Karlsruhe.xml"
bad_numbers, count = process_phone(data)

pprint.pprint(count)

{'Expected_format': 648,
 'Whitespace_problem': 202,
 'no_country_code': 23,
 'numbers_only': 285,
 'other': 161}


The output above shows the count of phone numbers that either met or did not meet the following expected format:<br><br>
"(Country_code: +49) (local_code: any combination of numbers) (phone_number: any combination of numbers)"<br><br>
From the results summary one can see that the majority of 648 data entries meet this specific format.Some 202 phone numbers only have a whitespace problem which can be cleaned up programmatically (See below). 23 numbers do not contain country codes and unfortunately, I can do little about that but going through them manually and gues the country codes. 285 datapoints are only a sequence of numbers. Cleaning these datapoints requires manual work as well, since it is hard to separate all of them (into country_code) : (local-code) : (phone-number) programmatically assuming there are various different country and local codes potentially present in the data. However, I will try to extract all phone numbers with local-code from Karlsruhe and bring them in the expected format. Finally there are some 161 datapoints which I have labeled "other" meaning that the format of these phone numbers follows various different patterns. Below I have printed out some exmaples of these "other" formats. 

In [2]:
z=0
for e in bad_numbers:
    if z == 5:
        break
    else:
        pprint.pprint(e)
        z+=1

'+49 /(0) 721 925 - 5700'
'+49 (0)721 86 48 97 84'
'+49 (0) 721 23 280'
'yes'
'+49721484040'


From the output above, one can see that the format of these numbers varies a lot. Hence, in order to clean these datapoints I could either find some more patterns in the formatting or go through them manually.<br> The output also shows the value "yes" which is not a phone-number at all. Investigating this datapoint, I found that a wrong OSM key was included in the analysis which should be excluded in following investigations.

#### Inconsistent name for a local company
Another problem which I encountered in the dataset is an inconsistent use of different names for the same company. The company "Aldi" is a large German supermarket chain with various stores all over Karlsruhe. With my analysis I could find some problems with the inconsistent use of the company's name in the OSM dataset.

In [3]:
def audit_company_name(element, count):
    company_pattern = re.compile(r"^(Aldi)(\s|$)",re.I)
    if element.attrib["k"] in ["name", "brand","operator"]:
        if company_pattern.search(element.attrib["v"]):
            name = element.attrib["k"] + "=" + element.attrib["v"]
            count["Anzahl"] += 1
            return name
    
def process_company_audit(filename,tags=("tag")):
    names = set()
    count = {"Anzahl":0}
    for element in get_element(filename, tags):
        name = audit_company_name(element,count)
        names.add(name)
    return names, count

pprint.pprint(process_company_audit(data))

(set([None,
      u'brand=Aldi S\xfcd',
      'name=ALDI',
      'name=Aldi',
      u'name=Aldi S\xfcd',
      'operator=ALDI GmbH & Co. KG, Rastatt',
      'operator=Aldi',
      u'operator=Aldi S\xfcd']),
 {'Anzahl': 22})


The output above shows all different kinds of ways how the company's name is being spelled in the OSM dataset. Below I will remove this inconsistency and name the company consistently throughout the dataset.

#### Inconsistent date formats in key = "start_date"

Looking at the date formats used for the OSM key "start_date" I found that quite a lot of the data entries do not follow the expected format "YYYY-MM-DD".

In [4]:
def audit_date(element,count):
    date_pattern = re.compile(r"^([0-9]{4,4}\-[0-9]{2,2}\-[0-9]{2,2}$)")
    if element.attrib["k"] == "start_date":
        if date_pattern.search(element.attrib["v"]):
            count["Expected"]+=1
        else:
            count["other"]+=1
            return element.attrib["v"]
            
def process_date(filename,tags=("tag")):
    dates = set()
    count = {"Expected":0,"other":0}
    for element in get_element(filename, tags):
        date = audit_date(element,count)
        dates.add(date)
    return dates, count

dates, count = process_date(data)
pprint.pprint(count)

{'Expected': 480, 'other': 1158}


The output above shows that a majority of 1.158 datapoints do not have the expected format. However, looking at some examples with an unexpected date format I could quickly see a pattern.

In [ ]:
<tag k="start_date" v="1908" />
<tag k="start_date" v="2012-06" />
<tag k="start_date" v="~2015" />
<tag k="start_date" v="09/2014" />

From the examples above, it is clear that the problem is not necessarily caused by an inconsistent use of different date formats (even though this is potentially a problem in a few cases) but rather a problem of missing data or missing information about the exact "start_date" including day and month information. Hence, cleaning the date by forcing all datapoints into a standardized format does not improve the data quality in the sense that we get more accurate information. Therefore, I assume that the value-add of cleaning the "start_date" data is negligible in this case and thus, I will not clean this datafield.

### Migrating the OSM dataset to CSV

After having identified some problems with the OSM dataset, I have first migrated the OSM dataset to a CSV files before starting to clean the data. The csv dataset does not include any unusual OSM keys as mentioned earlier.

### Data Cleaning
Having already identified causes for some data quality problems in the OSM dataset, I can now continue with defining operations to clean the dataset and get rid of the encountered data quality issues.

#### Programmatically cleaning phone numbers

The following operation updates all phone numbers with whitespace problems, such that: “+49 721 133-5630” becomes: “+49 721 1335630”, for instance. Also phone numbers that contain only digits and the local code "0721" at the beginning have been cleaned. For the "other" cases mentioned above, automatic cleaning is not feasible, hence these types of phones numbers are not changed.

In [5]:
def update_number(number,count):
    expected_pattern = re.compile(r"^(\+49)\s[0-9]*\s[0-9]*$",re.I)
    whitespace_end = re.compile(r"^(\+49)\s[0-9]*\s[0-9\s|\-]*$")
    whitespace_start = re.compile(r"^(\+49)[\-|\s*][0-9]*[\-|\s*][0-9]*$")
    numbers_only = re.compile(r"^[0-9]*$",re.I)
    
    number = re.sub(' +',' ',number)
    
    if whitespace_end.search(number) and not expected_pattern.search(number):
        pattern = re.search(r"^(.+?)\s(.+?)\s(.+?)$",number)
        country_code = pattern.group(1)
        local_code = pattern.group(2)
        dial = "".join(("".join(pattern.group(3).split())).split("-"))
        new = country_code + " " + local_code + " " + dial
        count["cleaned"] += 1         
        return new
        
    elif whitespace_start.search(number) and not expected_pattern.search(number):
        count["cleaned"] += 1          
        return re.sub('-',' ',number) 
        
    elif numbers_only.search(number):
        start_local_code = re.compile(r"^(0721)")
        
        if start_local_code.search(number):
            no_pattern = re.search(r"^(0721)(.+?)$",number)
            dial = no_pattern.group(2)
            number = "+49 " + "721 " + dial
            count["cleaned"] += 1             
            return number
    else:
        return number
       
# update_number: Takes as input a list of csv filenames. Reads in the csv files, and cleans
        # for all keys that contain "phone" the value using update_number and saves the cleaned csv.
        
def clean_phone_numbers(files):    
    count = {"cleaned":0}   
    for datei in files:
        filename = datei
        tempfile = NamedTemporaryFile(delete=False)
        
        with open(filename,"rb") as csvfile,tempfile:
            reader = csv.DictReader(csvfile,delimiter=",")
            writer = csv.DictWriter(tempfile,delimiter=",",fieldnames = reader.fieldnames)
            writer.writeheader()
            for row in reader:
                if "phone" in row['key']:
                    row['value'] = update_number(row['value'],count)
                writer.writerow(row)
        shutil.move(tempfile.name, filename)
    return count

Using this cleaning operation a total of 220 phone numbers have been cleaned and changed into a consistent formatting.

In [ ]:
{'cleaned': 220}

#### Programmatically cleaning inconsistent company naming

In order to have a consistent naming for all Aldi branches in Karlsruhe, I have defined the following operation which forces all firm names to be the same, such that: "ALDI GmbH & Co. KG, Rastatt" becomes: "Aldi", for instance. 

In [ ]:
def update_firm_name(name,count):
    company_pattern = re.compile(r"^(Aldi)(\s|$)",re.I)    
    if company_pattern.search(name):
        name = "Aldi"        
        count["cleaned"] += 1 
        return name

A total of 22 datafields were cleand.

In [ ]:
{'cleaned': 22}

### Migrating the OSM dataset from CSV to a SQL database

After having cleaned the dataset and having updated the csv files, I migrated the OSM dataset to a SQL database. For adding the OSM dataset for Karlsruhe to a SQL database I have used [the schema provided by Stephen Welch](https://gist.github.com/swwelch/f1144229848b407e0a5d13fcb7fbbd6f).


### Data Overview & Data Exploration

To get an idea about my hometown and some facts and figures, I will explore the OSM dataset in the following section using SQL queries.

#### File sizes

In [ ]:
Karlsruhe.osm ......... 161 MB
Karlsruhe.db .......... 89 MB
nodes.csv ............. 49 MB
nodes_tags.csv ........ 5 MB
ways.csv .............. 6 MB
ways_tags.csv ......... 14 MB
ways_nodes.csv ......... 21 MB  

#### Number of nodes in Karlsruhe

In [ ]:
#SQL Query
SELECT COUNT(*) FROM nodes;

630960

#### Number of ways in Karlsruhe

In [ ]:
#SQL Query
SELECT COUNT(*) FROM ways;

111083

#### Top 10 contributing users to OSM Karlsruhe

In [ ]:
#SQL Query
SELECT e.user, COUNT(*) as num
   FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
   GROUP BY e.user
   ORDER BY num DESC
   LIMIT 10;

In [ ]:
ARWIE		201574
mapper999	171732
Blunauer	112203
leloop		32762
jlcod		19166
maction		14433
aluka		13564
drhzbg		13553
B-Rabbit	9074
Nakaner		8961

#### Most popular cuisine in Karlsruhe

In [ ]:
#SQL Query
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='cuisine'
GROUP BY nodes_tags.value
ORDER BY num DESC;

In [ ]:
italian		39
german		16
greek		16
regional	15
pizza		14
indian		11
thai		10
chinese		8
burger		5
turkish		5

#### Most common leisure facilities Karlsruhe

In [ ]:
#SQL Query
SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags
WHERE nodes_tags.key='leisure'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;

In [ ]:
playground		    134
pitch			    32
picnic_table		25
sports_centre		24
outdoor_seating		17
fitness_station		13
adult_gaming_centre	6
dance			    5
fitness_centre		5
sauna			    4

#### Number of nightclubs in Karlsruhe

In [ ]:
#SQL Query
SELECT COUNT(nodes_tags.id) FROM nodes_tags
	JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='nightclub') i
    	ON nodes_tags.id=i.id
WHERE nodes_tags.key='amenity';

22

#### Most popular tourism sites in Karlsruhe

In [ ]:
#SQL Query
SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags
   WHERE nodes_tags.key='tourism'
   GROUP BY nodes_tags.value
   ORDER BY num DESC
   LIMIT 10;

In [ ]:
information			217
artwork				102
hotel				35
picnic_site			14
attraction			10
museum				9
viewpoint			6
guest_house			3
bed_and_breakfast	1
hostel				1


#### Number of historic places in Karlsruhe

In [ ]:
#SQL Query
SELECT COUNT(nodes_tags.id) FROM nodes_tags
	JOIN (SELECT DISTINCT(id) FROM nodes_tags) i
    	ON nodes_tags.id=i.id
WHERE nodes_tags.key='historic';

368

### Conclusion & additional ideas

Wrangling the OSM dataset for my hometown was an interesting experience, since the open-source concept of OSM makes an exiting data source available to everybody. At the end of the Data Wrangling process of auditing, cleaning and migrating the data, I conducted some data exploration and found some interesting facts & figures about Karlsruhe.<br>
However, data quality concerns for this open source content remain. This is because it is fairly hard to assess the completeness of the dataset without a benchmark. It is however, very likely that OSM data is incomplete. Hence, this is a problem which should always be kept in mind when dealing with OSM data. In order to improve the completeness assessment of the OSM data, one could compare its information to datasets of other providers such as Google Maps. However, Google data cannot be accessed as easy as OSM data since it is not open-source.